In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import TruncatedSVD
from scipy.spatial import distance
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model, load_model
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras.utils import plot_model
from scipy.spatial.distance import cdist, cosine
import matplotlib.pyplot as plt
from tensorboard.plugins import projector
import joblib

2023-02-21 16:23:13.883831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 16:23:16.432255: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-21 16:23:20.545630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-21 16:23:20.545896: W tensorfl

In [ ]:
player_embedding = pd.read_csv('player_embedding.csv').drop(['Unnamed: 0'], axis=1).iloc[:,3:]
df = pd.read_csv('final_data/fifa_cleaned_all_columns.csv').drop(['Unnamed: 0'], axis=1)
player_details = df[['club_name','short_name','overall','season']]

In [ ]:
def get_position(club_pos):
    pos = club_pos.split(",")[0]
    return pos
position = df.apply(lambda x: get_position(x['player_positions']), axis=1)
position

0          CF
1          LW
2          RM
3          ST
4          CM
         ... 
126283    CDM
126284     CM
126285     CM
126286     ST
126287    CAM
Length: 126288, dtype: object

In [ ]:
player_embedding

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
0,3.265667,0.757148,1.709361,0.0,1.870791,0.000000,0.0,0.0,0.932092,0.0,1.050203,0.779398,0.0,0.0,3.547777,2.535839,0.914676,1.284182,1.337910,1.294660
1,3.758625,1.209742,1.564240,0.0,0.474581,0.460083,0.0,0.0,0.521481,0.0,1.467055,0.986364,0.0,0.0,2.773616,2.456737,0.486339,0.508786,0.510534,1.164134
2,2.898894,1.266350,1.875545,0.0,1.957869,0.040286,0.0,0.0,0.886252,0.0,0.966359,0.767043,0.0,0.0,3.300267,2.276494,1.297398,0.690070,0.657781,1.098054
3,3.622704,1.384349,1.968731,0.0,0.689646,0.611457,0.0,0.0,0.577605,0.0,0.934327,0.825177,0.0,0.0,2.274490,1.901708,0.385015,1.425968,0.401931,0.913824
4,2.284001,1.387072,2.395956,0.0,1.186806,0.054275,0.0,0.0,1.419867,0.0,0.816368,1.042094,0.0,0.0,2.159883,2.955362,0.189032,0.535729,0.991364,1.050213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126283,0.463187,0.528406,0.896661,0.0,0.989502,1.371228,0.0,0.0,0.669588,0.0,0.600177,0.927473,0.0,0.0,0.705462,1.019581,0.764323,0.865071,0.617943,0.469705
126284,0.459426,0.739470,0.860212,0.0,0.998605,1.291639,0.0,0.0,0.672569,0.0,0.471460,0.524431,0.0,0.0,0.756158,1.039984,0.670674,0.754113,0.496114,0.379089
126285,0.583554,0.534359,0.883899,0.0,1.025931,1.386753,0.0,0.0,0.701014,0.0,0.592992,0.746789,0.0,0.0,0.712209,0.987118,0.721504,1.002327,0.550613,0.527887
126286,0.524298,0.523198,0.106561,0.0,1.139934,1.555437,0.0,0.0,0.601059,0.0,0.505016,0.555318,0.0,0.0,1.317611,0.408143,0.491084,1.087960,0.779156,0.582550


In [ ]:
#Combined player position in cell 6, player details in cell 18, and player embedding into one df
#Create a key for each row using club_name and season for team embedding

player_embedding_combined = pd.concat([player_details, position.rename('best_pos'), player_embedding], axis=1)
player_embedding_combined['key'] = player_embedding_combined['club_name'].astype(str) + player_embedding_combined['season'].astype(str)
player_embedding_combined = player_embedding_combined.sort_values(by=['overall'], ascending=False)

#Create a column to classify players into defenders and non-defenders based on position
#Create a key for each row using club_name, season, and position classification for team embedding
def determine_defender(position):
    if ((position == 'LWB') or (position == 'LB') or (position == 'LCB') or (position == 'CB') or (position == 'RCB') or (position == 'RB') or (position == 'RWB')):
        return "D"
    else:
        return "ND"
player_embedding_combined['defender'] = player_embedding_combined.apply(lambda x: determine_defender(x['best_pos']), axis=1)
player_embedding_combined['player_key'] = player_embedding_combined['key'].astype(str) + player_embedding_combined['defender'].astype(str)
player_embedding_combined.head()

,club_name,short_name,overall,season,best_pos,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,key,defender,player_key
43907,Real Madrid CF,Cristiano Ronaldo,94,1718,LW,4.122661,1.222620,1.456111,0.0,0.988493,...,0.0,2.423369,2.852642,0.299847,0.576179,0.748574,1.174775,Real Madrid CF1718,ND,Real Madrid CF1718ND
59842,Juventus,Cristiano Ronaldo,94,1819,ST,4.006461,1.190789,1.405771,0.0,0.936381,...,0.0,2.313507,2.750958,0.377921,0.566794,0.707652,1.149878,Juventus1819,ND,Juventus1819ND
59843,FC Barcelona,L. Messi,94,1819,CF,3.385196,0.747430,2.103769,0.0,2.436939,...,0.0,3.431670,2.777207,0.796833,1.467861,1.355075,1.453514,FC Barcelona1819,ND,FC Barcelona1819ND
14380,FC Barcelona,L. Messi,94,1516,RW,3.246406,0.807521,1.621410,0.0,2.192120,...,0.0,3.447260,2.568624,0.887763,1.365363,1.271545,1.224468,FC Barcelona1516,ND,FC Barcelona1516ND
28304,Real Madrid CF,Cristiano Ronaldo,94,1617,LW,4.126410,1.259415,1.360926,0.0,0.948438,...,0.0,2.421443,2.830358,0.374458,0.605529,0.769751,1.149873,Real Madrid CF1617,ND,Real Madrid CF1617ND


In [ ]:
player_embedding_combined_D = player_embedding_combined[player_embedding_combined['defender']=="D"]
player_embedding_combined_D = player_embedding_combined_D.groupby('player_key').head(8).reset_index(drop=True).sort_values(by=['overall'], ascending=False)

player_embedding_combined_ND = player_embedding_combined[player_embedding_combined['defender']=="ND"]
player_embedding_combined_ND = player_embedding_combined_ND.groupby('player_key').head(12).reset_index(drop=True).sort_values(by=['overall'], ascending=False)

player_embedding_combined_D_ND = pd.concat([player_embedding_combined_D, player_embedding_combined_ND])
player_embedding_combined_D_ND = player_embedding_combined_D_ND.reset_index(drop=True)

In [ ]:
player_embedding_combined_D_ND[player_embedding_combined_D_ND['key'].str.contains('Real Madrid CF1516')]

,club_name,short_name,overall,season,best_pos,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,key,defender,player_key
43,Real Madrid CF,Sergio Ramos,87,1516,CB,2.124804,1.004520,2.782751,0.0,0.150618,...,0.0,0.727187,3.042945,1.414106,0.422430,0.595326,0.596105,Real Madrid CF1516,D,Real Madrid CF1516D
168,Real Madrid CF,Pepe,84,1516,CB,1.780926,0.879123,2.394005,0.0,0.651587,...,0.0,0.472614,2.636265,1.413166,1.645054,0.638638,0.748284,Real Madrid CF1516,D,Real Madrid CF1516D
322,Real Madrid CF,Marcelo,83,1516,LB,2.201063,1.441615,2.990856,0.0,1.735173,...,0.0,1.852750,2.953912,2.023668,0.549884,0.727022,0.715922,Real Madrid CF1516,D,Real Madrid CF1516D
428,Real Madrid CF,R. Varane,82,1516,CB,1.715679,0.281394,2.263497,0.0,0.749940,...,0.0,0.299836,2.450104,1.380157,1.651759,0.636851,0.914882,Real Madrid CF1516,D,Real Madrid CF1516D
694,Real Madrid CF,Carvajal,81,1516,RB,1.710597,1.299569,2.356246,0.0,0.531332,...,0.0,0.934706,2.910544,1.234861,0.434099,0.886419,0.469652,Real Madrid CF1516,D,Real Madrid CF1516D
819,Real Madrid CF,Danilo,80,1516,RB,2.315021,1.163128,2.015605,0.0,0.857278,...,0.0,0.825256,2.858598,1.138036,0.487195,0.339348,0.495871,Real Madrid CF1516,D,Real Madrid CF1516D
1273,Real Madrid CF,Arbeloa,79,1516,RB,1.647259,0.852351,2.123963,0.0,0.659760,...,0.0,0.358648,2.505687,1.217146,0.974281,0.607974,0.564623,Real Madrid CF1516,D,Real Madrid CF1516D
4384,Real Madrid CF,Nacho Fernández,75,1516,CB,1.205409,0.539433,2.276125,0.0,0.412004,...,0.0,0.600022,2.075383,1.417974,0.879538,0.616044,0.499204,Real Madrid CF1516,D,Real Madrid CF1516D
39985,Real Madrid CF,Cristiano Ronaldo,93,1516,LW,3.953484,1.234891,1.221004,0.0,0.894786,...,0.0,2.346539,2.661075,0.461978,0.602689,0.593610,1.076918,Real Madrid CF1516,ND,Real Madrid CF1516ND
40123,Real Madrid CF,L. Modrić,87,1516,CM,2.269099,1.143761,2.227264,0.0,1.282948,...,0.0,1.606233,3.258205,0.348104,0.528267,0.569568,0.815508,Real Madrid CF1516,ND,Real Madrid CF1516ND


In [ ]:
#Flatten the df with 20 players to a single row with (20 * number of dimensions in player embeddings) columns
team_season_set = set(player_embedding_combined['key'].tolist())
all_teams_df = pd.DataFrame()

for team in team_season_set:
    team_df = player_embedding_combined_D_ND[player_embedding_combined_D_ND['key']==team]
    team_df = team_df.drop(['best_pos','club_name','short_name','overall','season','key','defender','player_key'], axis=1)
    team_row = team_df.stack().reset_index().drop(['level_0','level_1'],axis=1).transpose()
    all_teams_df = pd.concat([all_teams_df, team_row])
#display(all_teams_df)

#player_embedding_combined_D_ND[player_embedding_combined_D_ND['key']=='Cork City1718']
all_teams_df['team'] = list(team_season_set)
all_teams_df = all_teams_df.dropna()
teams = all_teams_df['team']

#This cell and the cell below are optional (this is to add the league level to the team embeddings as a seaprate feature)
league_level_info = df[['club_name', 'season', 'league_level']]
league_level_info['key'] = league_level_info['club_name'].astype(str) + league_level_info['season'].astype(str)
league_level_info = league_level_info.drop(['club_name','season'],axis=1)
league_level_info = league_level_info.rename(columns={"key": "team"})
league_level_info['league_level'] = league_level_info['league_level']/5
league_level_info = league_level_info.drop_duplicates()

all_teams_df = all_teams_df.merge(league_level_info, on="team", how="left")
all_teams_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,team,league_level
0,1.212532,0.661989,2.436652,0.0,0.959036,1.704753,0.0,0.0,1.059886,0.0,...,0.0,0.0,1.212796,1.270461,0.668045,1.028716,0.565971,0.610142,Zagłębie Lubin2021,0.2
1,1.276283,0.546805,2.360179,0.0,0.650562,1.616852,0.0,0.0,0.740669,0.0,...,0.0,0.0,0.936555,1.672078,0.710642,0.922031,0.545203,0.685204,Derby County1920,0.4
2,1.341418,1.037563,2.507438,0.0,0.893255,1.596647,0.0,0.0,1.244697,0.0,...,0.0,0.0,1.316056,1.583573,0.618122,1.003232,0.497168,0.694642,Dijon FCO2122,0.4
3,1.507499,1.123514,1.980241,0.0,0.435500,1.291182,0.0,0.0,1.077614,0.0,...,0.0,0.0,0.881724,1.155251,0.604587,0.895576,0.407969,0.502002,Barnsley1718,0.4
4,1.899354,1.089398,2.346075,0.0,1.944487,0.626924,0.0,0.0,0.115714,0.0,...,0.0,0.0,1.713100,1.345427,1.580029,0.311806,0.466192,0.811964,CF Monterrey1920,0.2


In [ ]:
all_teams_df = all_teams_df.drop(['team'], axis=1)
teamscaler = MinMaxScaler()
X = teamscaler.fit_transform(all_teams_df)
y = all_teams_df['league_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
def train_large_nn(emb_dim = 50, epochs=100, batch_size=32):
    input = tf.keras.layers.Input(shape=X_train.shape[1:])

    encoder = tf.keras.layers.Dense(X_train.shape[1:][0], activation='relu')(input)
    encoder = tf.keras.layers.Dense(375, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(350, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(325, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(300, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(275, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(250, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(225, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(200, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(175, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(150, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(125, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(100, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(75, activation='relu')(encoder)
    embedding = tf.keras.layers.Dense(emb_dim, activation='relu',name='embedding')(encoder)

    decoder = tf.keras.layers.Dense(75, activation='relu')(embedding)
    decoder = tf.keras.layers.Dense(100, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(125, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(150, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(175, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(200, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(225, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(250, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(275, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(300, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(325, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(350, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(375, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')(decoder),

    autoencoder = Model(input, decoder)
    autoencoder.compile(loss='mae', optimizer='adam')
    callback = EarlyStopping(monitor='loss', patience=3)

    history = autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, #shuffle=False, 
                    callbacks=[callback], validation_data = (X_test, X_test))
    return history

In [ ]:
def train_small_nn(emb_dim = 50, epochs=100, batch_size=32):
    input = tf.keras.layers.Input(shape=X_train.shape[1:])

    encoder = tf.keras.layers.Dense(X_train.shape[1:][0], activation='relu')(input)
    encoder = tf.keras.layers.Dense(350, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(300, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(250, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(200, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(150, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(100, activation='relu')(encoder)
    embedding = tf.keras.layers.Dense(emb_dim, activation='relu',name='embedding')(encoder)

    decoder = tf.keras.layers.Dense(100, activation='relu')(embedding)
    decoder = tf.keras.layers.Dense(150, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(200, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(250, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(300, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(350, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')(decoder),
    
    autoencoder = Model(input, decoder)
    autoencoder.compile(loss='mae', optimizer='adam')
    callback = EarlyStopping(monitor='loss', patience=3)

    history = autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, #shuffle=False, 
                    callbacks=[callback], validation_data = (X_test, X_test))
    return history

In [ ]:
tune_dim = [30,40,50]
results = []
for dim in tune_dim:
    print('Start Training with Latent Space Dimension for Large NN: ' + str(dim) + '\n')
    results.append(train_large_nn(dim))
    print('Completed Training with Latent Space Dimension for Large NN: ' + str(dim) + '\n\n')
    
for dim in tune_dim:
    print('Start Training with Latent Space Dimension for Small NN: ' + str(dim) + '\n')
    results.append(train_small_nn(dim))
    print('Completed Training with Latent Space Dimension for Small NN: ' + str(dim) + '\n\n')

Start Training with Latent Space Dimension for Large NN: 30



2023-02-21 16:25:15.102570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 16:25:15.532487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 16:25:15.534446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 16:25:15.563025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/100
108/108 [==============================] - 8s 9ms/step - loss: 0.1145 - val_loss: 0.0886
Epoch 2/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0895 - val_loss: 0.0883
Epoch 3/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0891 - val_loss: 0.0876
Epoch 4/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0881 - val_loss: 0.0858
Epoch 5/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0859 - val_loss: 0.0843
Epoch 6/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0852 - val_loss: 0.0841
Epoch 7/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0849 - val_loss: 0.0845
Epoch 8/100
108/108 [==============================] - 1s 7ms/step - loss: 0.0848 - val_loss: 0.0839
Epoch 9/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0848 - val_loss: 0.0841
Epoch 10/100
108/108 [==============================] - 1s 6ms/step - loss: 0.0846 - val_lo

In [ ]:
svd4 = TruncatedSVD(n_components=30, n_iter=20, random_state=42)
all_teams_reduced30 = svd4.fit_transform(teamscaler.fit_transform(all_teams_df))
svd5 = TruncatedSVD(n_components=40, n_iter=20, random_state=42)
all_teams_reduced40 = svd5.fit_transform(teamscaler.fit_transform(all_teams_df))
svd6 = TruncatedSVD(n_components=50, n_iter=20, random_state=42)
all_teams_reduced50 = svd6.fit_transform(teamscaler.fit_transform(all_teams_df))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklea

In [ ]:
print('For Teams SVD Hyperparameter Tuning: ')

d = teamscaler.fit_transform(all_teams_df).copy()
d = pd.DataFrame(d)

c = pd.DataFrame(svd4.inverse_transform(all_teams_reduced30))
e = abs(c-d).mean().mean()
print('MAE of 30 Components: ' + str(e))

c = pd.DataFrame(svd5.inverse_transform(all_teams_reduced40))
e = abs(c-d).mean().mean()
print('MAE of 40 Components: ' + str(e))

c = pd.DataFrame(svd6.inverse_transform(all_teams_reduced50))
e = abs(c-d).mean().mean()
print('MAE of 50 Components: ' + str(e))

For Teams SVD Hyperparameter Tuning: 
MAE of 30 Components: 0.06695705574612534
MAE of 40 Components: 0.062094372041886614
MAE of 50 Components: 0.05822202128957278


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
print('For Teams Large NN Autoencoder Hyperparameter Tuning: ')
print('MAE of Dimension 30: ' + str(min(results[0].history['val_loss'])))
print('MAE of Dimension 40: ' + str(min(results[1].history['val_loss'])))
print('MAE of Dimension 50: ' + str(min(results[2].history['val_loss'])))

print('\nFor Teams Small NN Autoencoder Hyperparameter Tuning: ')
print('MAE of Dimension 30: ' + str(min(results[3].history['val_loss'])))
print('MAE of Dimension 40: ' + str(min(results[4].history['val_loss'])))
print('MAE of Dimension 50: ' + str(min(results[5].history['val_loss'])))

For Teams Large NN Autoencoder Hyperparameter Tuning: 
MAE of Dimension 30: 0.08352683484554291
MAE of Dimension 40: 0.08350678533315659
MAE of Dimension 50: 0.08354625850915909

For Teams Small NN Autoencoder Hyperparameter Tuning: 
MAE of Dimension 30: 0.08259359002113342
MAE of Dimension 40: 0.0824931263923645
MAE of Dimension 50: 0.08271608501672745


In [ ]:
train_small_nn(emb_dim = 50, epochs=200, batch_size=32)

Epoch 1/200
108/108 [==============================] - 1s 6ms/step - loss: 0.1073 - val_loss: 0.0886
Epoch 2/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0896 - val_loss: 0.0880
Epoch 3/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0890 - val_loss: 0.0873
Epoch 4/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0865 - val_loss: 0.0845
Epoch 5/200
108/108 [==============================] - 0s 5ms/step - loss: 0.0850 - val_loss: 0.0842
Epoch 6/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0847 - val_loss: 0.0840
Epoch 7/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0846 - val_loss: 0.0838
Epoch 8/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0845 - val_loss: 0.0843
Epoch 9/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0844 - val_loss: 0.0838
Epoch 10/200
108/108 [==============================] - 0s 4ms/step - loss: 0.0843 - val_lo